# 📘 Critério de Jury – Guia de Estudos Interativo

### ✍️ Autoria
- **Prof. Luiz Carlos de Freitas Júnior**  
- Universidade São Francisco – Cursos de Engenharia  
- Ano/Semestre: 2025/2  

---

### 🎯 Objetivos do Colab
- Disponibilizar um ambiente **interativo** para o estudo do **Critério de Jury**.  
- Auxiliar os estudantes a compreenderem e aplicarem o método em **sistemas discretos**.  
- Oferecer ferramentas práticas para:  
  - Construir a **Tabela de Jury** (famílias A, B, C, …).  
  - Realizar a **redução até grau 2**.  
  - Verificar **pré-condições** de estabilidade.  
  - Obter um **veredito automático** (Estável ❇️ / Instável ❌).  
  - **Exportar resultados** (CSV/Markdown) para relatórios acadêmicos.  

---

### 📚 Conexão com a Disciplina
- Parte integrante do conteúdo de **Controle Digital**.  
- Relacionado ao **Plano de Atividades 2025/2**, competência 4: *Analisar a estabilidade de sistemas discretos*.  
- Útil para resolução de **exercícios de prova** e **trabalhos práticos**.

---

### ✅ Como usar este Colab
1. Leia o fluxograma e o glossário.  
2. Insira os coeficientes do polinômio característico nos blocos indicados.  
3. Execute célula por célula, acompanhando os resultados.  
4. Compare os resultados da **redução** e das **famílias**.  
5. Use a célula de **exportação** para gerar arquivos de apoio ao seu relatório.  

---

> 🔔 **Observação:** Este material é de uso acadêmico, aberto para todos os estudantes da disciplina.  
> Sinta-se à vontade para explorar, modificar e reutilizar nos seus estudos.


# 📘 Roteiro de Estudos – Critério de Jury em Python (Colab Didático)

Este notebook foi criado para apoiar o estudo do **Critério de Jury**, ferramenta clássica para análise de **estabilidade de sistemas discretos**.  
Aqui você encontrará a lógica do código em forma de fluxograma, um glossário das bibliotecas utilizadas e um glossário dos principais comandos implementados.

---

## 🔹 1. Fluxograma da lógica do Colab

```mermaid
flowchart TD
    A[Entrada: coeficientes do polinômio] --> B[Checagem das pré-condições]
    B -->|Falha| X[Veredito = ❌ Instável]
    B -->|Passa| C[Construção da Tabela de Jury]
    C --> D[Redução do grau sucessivamente]
    D -->|Até grau 2| E[Polinômio quadrático final]
    E --> F[Veredito final: ✅ Estável ou ❌ Instável]
    F --> G[Exibição em DataFrame]
    G --> H[(Opcional) Exportação para CSV/Markdown]


# **Título e objetivos**

# Critério de Jury (Sistemas Discretos) — Colab Didático

**Objetivo:** permitir que você **avalie estabilidade** de sistemas discretos pelo **Critério de Jury**, trabalhando:
- Pré-condições do Jury;
- Construção da **Tabela de Jury**;
- Parada em **grau 2** (para análise/relato) e veredito claro (**Estável / Instável**);
- Exploração por **coeficientes** e por **raízes**;
- Exportação dos resultados.

> Relembrando: um sistema discreto é **estável** se **todas as raízes** do polinômio característico estiverem **dentro do círculo unitário** (|z| < 1).


# **Resumo teórico**

## Resumo teórico (Critério de Jury)

Dado o polinômio característico:
\[
P(z) = a_0 z^n + a_1 z^{n-1} + \cdots + a_n,\quad a_0>0,
\]
o Critério de Jury verifica estabilidade sem calcular explicitamente as raízes.

**Pré-condições (necessárias):**
1. \(P(1) > 0\)
2. \((-1)^n P(-1) > 0\)
3. \(|a_n| < a_0\)

**Tabela de Jury (idéia):**
- A partir dos coeficientes \(A^{(0)}\), formam-se sequências reduzindo o grau a cada etapa.
- Em cada etapa, exige-se \(|a_m| < a_0\) (coef. último vs. primeiro).
- O processo segue até reduzir para **grau 2** (três coeficientes) — aqui paramos para relatório — ou até **grau 1** no teste completo.

> Neste notebook, paramos em **grau 2** (como ferramenta de estudo). O veredito “Estável/Instável” é dado **até esse ponto**. Para concluir o critério completo, seguiria até grau 1.


# Bloco 1 — Imports e utilitários

In [ ]:
import numpy as np
import pandas as pd

def as_array(x):
    return np.asarray(x, dtype=float)

def coeffs_to_ascending(coeffs, order="desc"):
    """
    Converte coeficientes para ORDEM CRESCENTE DE ÍNDICE (A0..An).
    - order="desc": entrada é [a0, a1, ..., an] (potências decrescentes).
    - order="asc" : entrada já está [A0, A1, ..., An].
    """
    c = as_array(coeffs).tolist()
    if order == "desc":
        return c[::-1]
    elif order == "asc":
        return c
    raise ValueError('order deve ser "desc" ou "asc"')

def coeffs_to_descending(a_asc):
    """Volta para ordem decrescente de potência (a0..an)."""
    return as_array(a_asc).tolist()[::-1]


# Bloco 2 — Núcleo do Jury (para em grau 2)

In [ ]:
def jury_until_quadratic(coeffs, order="desc", eps=1e-12, normalize=True, verbose=True):
    """
    Executa o Jury reduzindo até grau 2.
    - coeffs: coeficientes do polinômio.
    - order : "desc" (padrão, a0..an) ou "asc" (A0..An).
    Retorna: dict com stable, reason, preconditions, table, final_quadratic, reached_deg2.
    """
    # normaliza formatos
    a_asc  = coeffs_to_ascending(coeffs, order=order)  # A0..An
    a_desc = coeffs_to_descending(a_asc)                # a0..an (decrescente)

    A0 = as_array(a_desc).copy()
    notes, stable = [], True
    reason, reached_deg2 = "Todas as condições satisfeitas até grau 2.", False

    if len(A0) < 3:
        raise ValueError("Grau ≥ 2.")

    # convenção a0>0
    if A0[0] < 0:
        A0 = -A0
        notes.append("Líder negativo — invertido para manter a0>0.")

    n = len(A0) - 1

    # pré-condições
    def _eval_desc(a, z):
        m = len(a) - 1
        return sum(a[k] * z**(m-k) for k in range(m+1))

    P1  = _eval_desc(A0,  1.0)
    Pm1 = _eval_desc(A0, -1.0)
    cond1 = P1 > 0
    cond2 = ((-1)**n) * Pm1 > 0
    cond3 = abs(A0[-1]) < A0[0]

    pre = {"P(1)>0": cond1, "(-1)^n P(-1)>0": cond2, "|a_n|<a_0": cond3,
           "P(1)": float(P1), "(-1)^n P(-1)": float(((-1)**n)*Pm1),
           "a0": float(A0[0]), "an": float(A0[-1])}

    if verbose:
        print("[Pré-condições]")
        print(f"  P(1)={P1:.6g} > 0 ? {cond1}")
        print(f"  (-1)^n P(-1)={(((-1)**n)*Pm1):.6g} > 0 ? {cond2}")
        print(f"  |a_n|<{A0[0]:.6g}  ? {cond3}")

    if not (cond1 and cond2 and cond3):
        return {"stable": False, "reason": "Falha nas pré-condições do Jury.",
                "preconditions": pre, "table": [A0.tolist()],
                "final_quadratic": None, "reached_deg2": False, "notes": notes}

    table = [A0.copy()]
    Ak = A0.copy()
    while len(Ak) > 3:
        a0_k, am_k = Ak[0], Ak[-1]
        if not (abs(am_k) + eps < abs(a0_k)):
            return {"stable": False,
                    "reason": f"Falha intermediária: |a_m| >= a_0 (|{am_k:.6g}| >= {a0_k:.6g}).",
                    "preconditions": pre, "table": [row.tolist() for row in table],
                    "final_quadratic": None, "reached_deg2": False, "notes": notes}
        Bk = Ak[::-1]
        Ck = a0_k * Ak - am_k * Bk
        Ak1 = Ck[:-1].copy()
        if normalize:
            if abs(Ak1[0]) <= eps:
                return {"stable": False, "reason": "Coeficiente líder ~0 após redução (degenerado).",
                        "preconditions": pre, "table": [row.tolist() for row in table],
                        "final_quadratic": None, "reached_deg2": False, "notes": notes}
            Ak1 = Ak1 / Ak1[0]
        Ak = Ak1
        table.append(Ak.copy())

    reached_deg2 = True
    return {"stable": True, "reason": reason, "preconditions": pre,
            "table": [row.tolist() for row in table],
            "final_quadratic": Ak.tolist(), "reached_deg2": reached_deg2, "notes": notes}



# Bloco 3 — Saída bonita (veredito + DataFrame)

In [ ]:
def verdict_pretty(res):
    """Imprime veredito + pré-condições de forma amigável."""
    print("\n=== Veredito ===")
    print("Estável? ", "✅ SIM" if res["stable"] else "❌ NÃO")
    print("Motivo:  ", res["reason"])
    if res.get("final_quadratic") is not None:
        print("Quadrático final [b0, b1, b2]:", res["final_quadratic"])

    pre = res["preconditions"]
    ok  = lambda v: "✓" if v else "✗"
    print("\n[Pré-condições]")
    print(f"  P(1)>0:              {ok(pre['P(1)>0'])}  (P(1)={pre['P(1)']:.6g})")
    print(f"  (-1)^n P(-1)>0:      {ok(pre['(-1)^n P(-1)>0'])}  ((-1)^n P(-1)={pre['(-1)^n P(-1)']:.6g})")
    print(f"  |a_n| < a_0:         {ok(pre['|a_n|<a_0'])}  (|an|={abs(pre['an']):.6g} < a0={pre['a0']:.6g})")

def jury_table_df(table_rows):
    """Cria DataFrame com a Tabela de Jury (formatado)."""
    maxlen = max(len(r) for r in table_rows)
    cols = [f"c{k}" for k in range(maxlen)]
    idx  = [f"A^{i}" for i in range(len(table_rows))]
    data = [r + [""]*(maxlen-len(r)) for r in table_rows]
    df = pd.DataFrame(data, index=idx, columns=cols)

    # Formatação (pandas novo: .map; fallback: .applymap)
    def _fmt(x):
        return f"{x:.6g}" if isinstance(x, (int, float)) and x != "" else x
    try:
        df = df.map(_fmt)
    except Exception:
        df = df.applymap(_fmt)
    return df

def export_table(table_rows, basename="jury_table"):
    """Salva CSV e Markdown da Tabela de Jury."""
    df = jury_table_df(table_rows)
    csv_path = f"{basename}.csv"
    md_path  = f"{basename}.md"
    df.to_csv(csv_path, index=True)
    with open(md_path, "w", encoding="utf-8") as f:
        f.write(df.to_markdown())
    print(f"Arquivos salvos:\n- {csv_path}\n- {md_path}")
    return csv_path, md_path


# Bloco 4 — Helpers de exibição

In [ ]:
def verdict_pretty(res):
    print("\n=== Veredito ===")
    print("Estável? ", "✅ SIM" if res["stable"] else "❌ NÃO")
    print("Motivo:  ", res["reason"])
    if res.get("final_quadratic") is not None:
        print("Quadrático final [b0, b1, b2]:", res["final_quadratic"])
    pre = res["preconditions"]; ok = (lambda v: "✓" if v else "✗")
    print("\n[Pré-condições]")
    print(f"  P(1)>0:              {ok(pre['P(1)>0'])}  (P(1)={pre['P(1)']:.6g})")
    print(f"  (-1)^n P(-1)>0:      {ok(pre['(-1)^n P(-1)>0'])}  ((-1)^n P(-1)={pre['(-1)^n P(-1)']:.6g})")
    print(f"  |a_n| < a_0:         {ok(pre['|a_n|<a_0'])}  (|an|={abs(pre['an']):.6g} < a0={pre['a0']:.6g})")

def jury_table_df(table_rows):
    maxlen = max(len(r) for r in table_rows)
    cols = [f"c{k}" for k in range(maxlen)]
    idx  = [f"A^{i}" for i in range(len(table_rows))]
    data = [r + [""]*(maxlen-len(r)) for r in table_rows]
    df = pd.DataFrame(data, index=idx, columns=cols)
    def _fmt(x): return f"{x:.6g}" if isinstance(x,(int,float)) and x != "" else x
    try:   df = df.map(_fmt)       # pandas novo
    except: df = df.applymap(_fmt) # fallback
    return df



# Bloco 5 — Famílias A, B, C, … até 3 elementos

In [ ]:
import string

def build_jury_families_until_len3(a_asc, as_dataframe=True, fmt_six=True):
    """
    Monta as famílias A, B, C, ... até surgir uma família com 3 elementos.
    ENTRADA: a_asc = [A0, A1, ..., An] (ordem CRESCENTE de índice).
    Regra X->Y (se X tem m elementos): Y_k = X0*Xk - X_{m-1}*X_{m-1-k}, k=0..m-2
    Gera duas linhas por família: direta (→) e reversa (←).
    """
    x = as_array(a_asc).tolist()
    if len(x) < 3:
        raise ValueError("Forneça pelo menos 3 coeficientes (grau ≥ 2).")

    rows, fam_labels = [], list(string.ascii_uppercase)
    fam_idx, curr = 0, x
    while True:
        fam = fam_labels[fam_idx]
        rows.append((f"{fam} (→)", curr.copy()))
        rows.append((f"{fam} (←)", curr[::-1].copy()))
        if len(curr) == 3:
            break
        m, x0, xm1 = len(curr), curr[0], curr[-1]
        nxt = [x0*curr[k] - xm1*curr[m-1-k] for k in range(m-1)]
        curr, fam_idx = nxt, fam_idx + 1

    if not as_dataframe:
        return rows

    max_len = max(len(r[1]) for r in rows)
    cols = [f"c{k}" for k in range(max_len)]
    data, idx = [], []
    for name, vals in rows:
        pad = vals + [""]*(max_len - len(vals))
        data.append(pad); idx.append(name)
    df = pd.DataFrame(data, index=idx, columns=cols)

    if fmt_six:
        def _fmt(v): return f"{v:.6g}" if isinstance(v,(int,float)) and v != "" else v
        try:   df = df.map(_fmt)
        except: df = df.applymap(_fmt)
    return df


# Bloco 6 — Entrada do usuário + execução unificada

In [ ]:
# ======= Entrada do usuário =======
# Ex.: "desc" -> [a0, a1, ..., an] (potências decrescentes)
#     "asc"  -> [A0, A1, ..., An] (constante até líder)
coeffs_user = [1, -3.1, 3.29, -1.457, 0.267, -0.01512]
order_user  = "desc"   # mude para "asc" se já vier A0..An

# ======= Jury (redução até grau 2) =======
res = jury_until_quadratic(coeffs_user, order=order_user, verbose=True)
verdict_pretty(res)
print("\n=== Tabela (redução até grau 2) ===")
display(jury_table_df(res["table"]).style.set_caption("Tabela de Jury — Redução"))

# ======= Famílias A, B, C, ... (até 3 elementos) =======
a_asc = coeffs_to_ascending(coeffs_user, order=order_user)
df_fams = build_jury_families_until_len3(a_asc, as_dataframe=True)
print("\n=== Famílias A, B, C, ... (até 3 elementos) ===")
display(df_fams.style.set_caption("Tabela de famílias (Jury)"))



[Pré-condições]
  P(1)=-0.01512 > 0 ? False
  (-1)^n P(-1)=9.12912 > 0 ? True
  |a_n|<1  ? True

=== Veredito ===
Estável?  ❌ NÃO
Motivo:   Falha nas pré-condições do Jury.

[Pré-condições]
  P(1)>0:              ✗  (P(1)=-0.01512)
  (-1)^n P(-1)>0:      ✓  ((-1)^n P(-1)=9.12912)
  |a_n| < a_0:         ✓  (|an|=0.01512 < a0=1)

=== Tabela (redução até grau 2) ===


,c0,c1,c2,c3,c4,c5
A^0,1,-3.1,3.29,-1.457,0.267,-0.01512



=== Famílias A, B, C, ... (até 3 elementos) ===


,c0,c1,c2,c3,c4,c5
A (→),-0.01512,0.267,-1.457,3.29,-3.1,1
A (←),1,-3.1,3.29,-1.457,0.267,-0.01512
B (→),-0.999771,3.09596,-3.26797,1.40726,-0.220128,
B (←),-0.220128,1.40726,-3.26797,3.09596,-0.999771,
C (→),0.951086,-2.78548,2.54785,-0.725425,,
C (←),-0.725425,2.54785,-2.78548,0.951086,,
D (→),0.378324,-0.800955,0.40257,,,
D (←),0.40257,-0.800955,0.378324,,,


# Bloco 7 — Exportação

In [ ]:
def export_table(table_rows, basename="jury_table"):
    maxlen = max(len(r) for r in table_rows)
    cols = [f"c{k}" for k in range(maxlen)]
    idx  = [f"A^{i}" for i in range(len(table_rows))]
    data = [r + [""]*(maxlen-len(r)) for r in table_rows]
    df = pd.DataFrame(data, index=idx, columns=cols)
    csv_path = f"{basename}.csv"; md_path = f"{basename}.md"
    df.to_csv(csv_path, index=True)
    with open(md_path, "w", encoding="utf-8") as f:
        f.write(df.to_markdown())
    print(f"Arquivos salvos:\n- {csv_path}\n- {md_path}")
    return csv_path, md_path

# Exporta a tabela da REDUÇÃO:
_ = export_table(res["table"], basename="jury_reducao")

# Exporta a tabela das FAMÍLIAS:
df_fams.reset_index().to_csv("jury_familias.csv", index=False)
with open("jury_familias.md", "w", encoding="utf-8") as f:
    f.write(df_fams.to_markdown())
print("Arquivos salvos também: jury_familias.csv, jury_familias.md")


Arquivos salvos:
- jury_reducao.csv
- jury_reducao.md
Arquivos salvos também: jury_familias.csv, jury_familias.md


# **Exercícios práticos**

## Exercícios práticos

1. **Grau 3 estável:** construa um polinômio cúbico que seja estável (dica: escolha 3 raízes com |z|<1), gere os coeficientes e aplique o Jury.
2. **Grau 4 com uma raiz instável:** escolha raízes \([0.2, 0.5, 1.1, 0.3]\), gere os coeficientes e avalie — o que acontece nas pré-condições?
3. **Efeito de normalização:** teste `normalize=False` na função `jury_until_quadratic` e comente diferenças numéricas.
4. **Comparação com o mapa de polos:** usando as raízes escolhidas, comente como o veredito do Jury se relaciona com a posição dos polos no plano-z.

> Registre as respostas nas células abaixo (texto). Use capturas de tela da Tabela de Jury quando necessário.

# Bloco 8 — Checagem de consistência

In [ ]:
# Bloco 8 — Checagem de consistência (normalizando PELO ÚLTIMO termo)

# pegue a ÚLTIMA FAMÍLIA na linha direta "(→)":
idx_last_direct = df_fams.index[-2] if df_fams.index[-2].endswith("(→)") else df_fams.index[-1]
fam_ultima_vals = df_fams.loc[idx_last_direct].tolist()

# filtra vazios e converte para float
fam_ultima = [float(x) for x in fam_ultima_vals if x != ""]
quad_reduc = res.get("final_quadratic")  # pode ser None se não chegou a grau 2

print("Quadrático (redução):", quad_reduc)
print("Última família (não normalizada):", fam_ultima)

# normaliza para que o ÚLTIMO termo seja 1 (maior índice)
if fam_ultima and abs(fam_ultima[-1]) > 1e-12:
    fam_norm = (np.array(fam_ultima, dtype=float) / fam_ultima[-1]).tolist()
    print("Última família (normalizada último=1):", [f"{x:.6g}" for x in fam_norm])
else:
    print("Não foi possível normalizar: último coeficiente ~ 0.")




Quadrático (redução): None
Última família (não normalizada): [0.378324, -0.800955, 0.40257]
Última família (normalizada último=1): ['0.939772', '-1.9896', '1']


# **Reflexão do aluno**

## Reflexão (respostas do aluno)

- **(1)** O que significa falhar em `P(1)>0` na sua interpretação?
- **(2)** Em qual etapa o seu exemplo instável “quebrou” (pré-condição ou etapa intermediária \(|a_m|<a_0\))?
- **(3)** Qual a vantagem de parar em **grau 2** para fins de relatório? O que faltaria para executar o **Jury completo**?
